# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察均值編碼的效果

# [作業重點]
- 仿造範例, 完成標籤編碼與均值編碼搭配邏輯斯迴歸的預測
- 觀察標籤編碼與均值編碼在特徵數量 / 邏輯斯迴歸分數 / 邏輯斯迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]) 

# 作業1
* 請仿照範例，將鐵達尼範例中的類別型特徵改用均值編碼實作一次

In [35]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

data_path = 'D:\\learning_ai\\D12\\'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [67]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 觀察鐵達尼生存預測中，均值編碼與標籤編碼兩者比較，哪一個效果比較好? 可能的原因是什麼?

In [31]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for col in df.columns:
    df_temp[col] = LabelEncoder().fit_transform(df[col])

train_X = df_temp[:train_num]
estimator = LogisticRegression()

start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(estimator, train_X, train_Y, cv = 5).mean()}')
print(f'time : {time.time()-start} sec')

shape : (891, 5)
score : 0.780004837244799
time : 0.03790116310119629 sec


In [74]:
# 均值編碼 + 邏輯斯迴歸

# 加回train_Y ('SalePrice')
data = pd.concat([df[:train_num], train_Y], axis = 1)
for col in df.columns:
    # 各類別欄位分取求目標欄位平均值
    col_mean = data.groupby([col])['Survived'].mean().reset_index()
    #給予標籤名稱 c, c_mean
    col_mean.columns = [col, f'{col}_mean']
    # 將均值編碼merge回data中
    data = pd.merge(data, col_mean, on = col, how = 'left')
    # 刪除原本c欄位(以用c_mean取代)
    data = data.drop(col, axis = 1)
#去除train_Y欄位  ('Survived')  
data = data.drop('Survived', axis = 1)
# 建立estimator
estimator = LogisticRegression()
# 儲存開始時間為start
start = time.time()

print(f'shape : {data.shape}')
print(f'score : {cross_val_score(estimator, data, train_Y, cv = 5).mean()}')
print(f'time : {time.time()-start} sec')

shape : (891, 5)
score : 1.0
time : 0.0279996395111084 sec


In [77]:
# nunique 找出各欄位有幾種類別
train_X.nunique()

Name        891
Sex           2
Ticket      681
Cabin       148
Embarked      4
dtype: int64

In [79]:
data.nunique()

Name_mean        2
Sex_mean         2
Ticket_mean      8
Cabin_mean       6
Embarked_mean    4
dtype: int64

標籤編碼較佳 <br>
均值編碼交叉驗證分數為1, 顯示已經overfitting了<br>
原因：Name和Ticket欄位類別數太多, 在對照目標值(Survived)時, 無法有效進行平均

In [80]:
#去除'Survived', 'Name', 'Ticket'
data = data.drop(['Name_mean', 'Ticket_mean'] , axis = 1)
# 建立estimator
estimator = LogisticRegression()
# 儲存開始時間為start
start = time.time()

print(f'shape : {data.shape}')
print(f'score : {cross_val_score(estimator, data, train_Y, cv = 5).mean()}')
print(f'time : {time.time()-start} sec')

shape : (891, 3)
score : 0.8350366889413987
time : 0.03889322280883789 sec
